In [2]:
import sys
sys.path.append("C:/Users/user/meepc")
import numpy as np
import pandas as pd
from models import Hankel,Corrhankel,Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

In [3]:
def calculate_fpr(y_actual, y_prediction):
    fp = 0
    tn = 0
    
    for i in range(len(y_actual)):
        if y_actual[i] == 0 and y_prediction[i] == 1:
            fp += 1
        elif y_actual[i] == 0 and y_prediction[i] == 0:
            tn += 1
    fpr= fp/(fp+tn)
    return fpr

In [4]:
train_normal = pd.read_csv('~/data/te/normal_training.csv')
train_attack = pd.read_csv('~/data/te/attack_training.csv')

In [5]:
sensors = [col for col in train_normal.columns if col not in ['faultNumber', 'simulationRun', 'sample','LABEL']]

In [6]:
scaler = StandardScaler()
X_normal = pd.DataFrame(index=train_normal.index, columns=sensors, data=scaler.fit_transform(train_normal[sensors]))
X_attack = pd.DataFrame(data=scaler.fit_transform(train_attack.loc[:,sensors]),index=train_attack.index,columns=sensors).reset_index().drop(columns=['index'])

In [7]:
hankel = Hankel()
corrhankel = Corrhankel()
lag = 360
stride = 0.5

In [8]:
corr_normal,nolag_normal = corrhankel.fit(X_normal.to_numpy(),lag,stride)

In [9]:
corr_attack,nolag_attack = corrhankel.fit(X_attack.to_numpy(),lag,stride)

In [10]:
df_test1 = pd.read_csv('~/data/te/normal_testing.csv')
df_test2 = pd.read_csv('~/data/te/attack_testing.csv')
df_test=pd.concat((df_test1,df_test2),axis=0)

# Epasad with 1 cluster and no threshold tuning (training attack included in test data)

In [11]:
test_combined = pd.concat((df_test,train_attack),axis=0)
X_test = pd.DataFrame(index=test_combined.index, columns=sensors, data=scaler.fit_transform(test_combined[sensors]))
corr_test,nolag_test = corrhankel.fit(X_test.to_numpy(),lag,stride)
Y_test = test_combined.loc[:,'LABEL']

In [12]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [13]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
fpr = []
for i,sens in enumerate(sensors):
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,tune=False,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,only_corr=True)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test,corr_test[:,i].reshape(nolag_test,len(X_test.columns)).T)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    fpr.append(calculate_fpr(y_actual,y_predicted))
    sensor_models.append(model)    

In [14]:
sensor_predicted = np.asarray(sensor_predicted)
y_predicted = np.any(sensor_predicted,axis=0).astype(int)
y_predicted

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [15]:
print("Accuracy ",accuracy_score(y_actual,y_predicted))
print("Precision ",precision_score(y_actual,y_predicted))
print("Recall ",recall_score(y_actual,y_predicted))
print("F1-score ",f1_score(y_actual,y_predicted))

Accuracy  0.9759036144578314
Precision  0.9759036144578314
Recall  1.0
F1-score  0.9878048780487805


In [16]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.7950648748841521 Median : 0.9759036144578314 Min : 0.21686746987951808 Max : 0.9759036144578314
precision - Mean: 0.981772695051431 Median : 0.9759036144578314 Min : 0.9724137931034482 Max : 1.0
recall -    Mean: 0.8080484330484331 Median : 1.0 Min : 0.19753086419753085 Max : 1.0
f1 -        Mean: 0.8472313699149073 Median : 0.9878048780487805 Min : 0.32989690721649484 Max : 0.9878048780487805


# Multiple clusters + No threshold tuning (training attack mixed in test data)

In [17]:
cluster_accuracy_1 = []
cluster_precision_1 = []
cluster_recall_1 = []
cluster_fscore_1 = []
cluster_fpr_1 = []
for k in range(2,8):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    fpr = []
    for i,sens in enumerate(sensors):
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k = k,tune=False,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,only_corr=True)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_test.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        fpr.append(calculate_fpr(y_actual,y_predicted))
        sensor_models.append(model)  
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int) 
    cluster_accuracy_1.append(accuracy_score(y_actual, y_predicted))
    cluster_precision_1.append(precision_score(y_actual,y_predicted))
    cluster_recall_1.append(recall_score(y_actual,y_predicted))
    cluster_fscore_1.append(f1_score(y_actual,y_predicted))
    cluster_fpr_1.append(calculate_fpr(y_actual,y_predicted))
    print('------Number of Clusters: ',k,'-----------') 
    print("Accuracy ",cluster_accuracy_1[-1])
    print("Precision ",cluster_precision_1[-1])
    print("Recall ",cluster_recall_1[-1])
    print("F1-score ",cluster_fscore_1[-1])
    print("False Positive Rate ",cluster_fpr_1[-1],"\n") 

------Number of Clusters:  2 -----------
Accuracy  0.9759036144578314
Precision  0.9759036144578314
Recall  1.0
F1-score  0.9878048780487805
False Positive Rate  1.0 

------Number of Clusters:  3 -----------
Accuracy  0.9759036144578314
Precision  0.9759036144578314
Recall  1.0
F1-score  0.9878048780487805
False Positive Rate  1.0 

------Number of Clusters:  4 -----------
Accuracy  0.9759036144578314
Precision  0.9759036144578314
Recall  1.0
F1-score  0.9878048780487805
False Positive Rate  1.0 

------Number of Clusters:  5 -----------
Accuracy  0.9759036144578314
Precision  0.9759036144578314
Recall  1.0
F1-score  0.9878048780487805
False Positive Rate  1.0 

------Number of Clusters:  6 -----------
Accuracy  0.9759036144578314
Precision  0.9759036144578314
Recall  1.0
F1-score  0.9878048780487805
False Positive Rate  1.0 

------Number of Clusters:  7 -----------
Accuracy  0.9759036144578314
Precision  0.9759036144578314
Recall  1.0
F1-score  0.9878048780487805
False Positive Rate

In [18]:
print("Accuracy Scores: ",cluster_accuracy_1)
print("Precision Scores: ",cluster_precision_1)
print("Recall Scores: ",cluster_recall_1)
print("F1 Scores: ",cluster_fscore_1)
print("False Positive Rates: ",cluster_fpr_1)

Accuracy Scores:  [0.9759036144578314, 0.9759036144578314, 0.9759036144578314, 0.9759036144578314, 0.9759036144578314, 0.9759036144578314]
Precision Scores:  [0.9759036144578314, 0.9759036144578314, 0.9759036144578314, 0.9759036144578314, 0.9759036144578314, 0.9759036144578314]
Recall Scores:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
F1 Scores:  [0.9878048780487805, 0.9878048780487805, 0.9878048780487805, 0.9878048780487805, 0.9878048780487805, 0.9878048780487805]
False Positive Rates:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


# Multiple clusters + No threshold tuning (No concat of training and test data)

In [19]:
X_test = pd.DataFrame(index=df_test.index, columns=sensors, data=scaler.fit_transform(df_test[sensors]))
corr_test,nolag_test = corrhankel.fit(X_test.to_numpy(),lag,stride)
Y_test = df_test.loc[:,'LABEL']

In [20]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [21]:
cluster_accuracy_2 = []
cluster_precision_2 = []
cluster_recall_2 = []
cluster_fscore_2 = []
cluster_fpr_2 = []
for k in range(2,8):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    fpr = []
    for sens in sensors:
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k=k,tune=False,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,only_corr=True)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_normal.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        fpr.append(calculate_fpr(y_actual,y_predicted))
        sensor_models.append(model)    
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    cluster_accuracy_2.append(accuracy_score(y_actual, y_predicted))
    cluster_precision_2.append(precision_score(y_actual,y_predicted))
    cluster_recall_2.append(recall_score(y_actual,y_predicted))
    cluster_fscore_2.append(f1_score(y_actual,y_predicted))
    cluster_fpr_2.append(calculate_fpr(y_actual,y_predicted))
    print('------Number of Clusters: ',k,'-----------') 
    print("Accuracy ",cluster_accuracy_2[-1])
    print("Precision ",cluster_precision_2[-1])
    print("Recall ",cluster_recall_2[-1])
    print("F1-score ",cluster_fscore_2[-1])
    print("False Positive Rate ",cluster_fpr_2[-1],"\n") 

------Number of Clusters:  2 -----------
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815
False Positive Rate  1.0 

------Number of Clusters:  3 -----------
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815
False Positive Rate  1.0 

------Number of Clusters:  4 -----------
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815
False Positive Rate  1.0 

------Number of Clusters:  5 -----------
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815
False Positive Rate  1.0 

------Number of Clusters:  6 -----------
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815
False Positive Rate  1.0 

------Number of Clusters:  7 -----------
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815
False Positive Rate

In [22]:
print("Accuracy Scores: ",cluster_accuracy_2)
print("Precision Scores: ",cluster_precision_2)
print("Recall Scores: ",cluster_recall_2)
print("F1 Scores: ",cluster_fscore_2)
print("False Positive Rates: ",cluster_fpr_2)

Accuracy Scores:  [0.9636363636363636, 0.9636363636363636, 0.9636363636363636, 0.9636363636363636, 0.9636363636363636, 0.9636363636363636]
Precision Scores:  [0.9636363636363636, 0.9636363636363636, 0.9636363636363636, 0.9636363636363636, 0.9636363636363636, 0.9636363636363636]
Recall Scores:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
F1 Scores:  [0.9814814814814815, 0.9814814814814815, 0.9814814814814815, 0.9814814814814815, 0.9814814814814815, 0.9814814814814815]
False Positive Rates:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


# Multiple clusters + Threshold tuning (No concat of training and test data)

In [23]:
cluster_accuracy_3 = []
cluster_precision_3 = []
cluster_recall_3 = []
cluster_fscore_3 = []
cluster_fpr_3 = []
for k in range(2,8):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    fpr = []
    for sens in sensors:
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k = k,tune=True,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,corr_attack=corr_attack[:,i].reshape(nolag_attack,len(X_attack.columns)).T,only_corr=True)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_normal.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        fpr.append(calculate_fpr(y_actual,y_predicted))
        sensor_models.append(model)    
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    cluster_accuracy_3.append(accuracy_score(y_actual, y_predicted))
    cluster_precision_3.append(precision_score(y_actual,y_predicted))
    cluster_recall_3.append(recall_score(y_actual,y_predicted))
    cluster_fscore_3.append(f1_score(y_actual,y_predicted))
    cluster_fpr_3.append(calculate_fpr(y_actual,y_predicted))
    print('------Number of Clusters: ',k,'-----------') 
    print("Accuracy ",cluster_accuracy_3[-1])
    print("Precision ",cluster_precision_3[-1])
    print("Recall ",cluster_recall_3[-1])
    print("F1-score ",cluster_fscore_3[-1])
    print("False Positive Rate ",cluster_fpr_3[-1],"\n")

------Number of Clusters:  2 -----------
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815
False Positive Rate  1.0 

------Number of Clusters:  3 -----------
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815
False Positive Rate  1.0 

------Number of Clusters:  4 -----------
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815
False Positive Rate  1.0 

------Number of Clusters:  5 -----------
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815
False Positive Rate  1.0 

------Number of Clusters:  6 -----------
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815
False Positive Rate  1.0 

------Number of Clusters:  7 -----------
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815
False Positive Rate

In [24]:
print("Accuracy Scores: ",cluster_accuracy_3)
print("Precision Scores: ",cluster_precision_3)
print("Recall Scores: ",cluster_recall_3)
print("F1 Scores: ",cluster_fscore_3)
print("False Positive Rates: ",cluster_fpr_3)

Accuracy Scores:  [0.9636363636363636, 0.9636363636363636, 0.9636363636363636, 0.9636363636363636, 0.9636363636363636, 0.9636363636363636]
Precision Scores:  [0.9636363636363636, 0.9636363636363636, 0.9636363636363636, 0.9636363636363636, 0.9636363636363636, 0.9636363636363636]
Recall Scores:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
F1 Scores:  [0.9814814814814815, 0.9814814814814815, 0.9814814814814815, 0.9814814814814815, 0.9814814814814815, 0.9814814814814815]
False Positive Rates:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
